In [1]:
#Install Packages
!pip install faiss-cpu
!pip install sentence-transformers

  Using cached faiss_cpu-1.10.0-cp313-cp313-win_amd64.whl.metadata (4.5 kB)
Using cached faiss_cpu-1.10.0-cp313-cp313-win_amd64.whl (13.7 MB)
  Using cached huggingface_hub-0.29.1-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached huggingface_hub-0.29.1-py3-none-any.whl (468 kB)
   ---------------------------------------- 0.0/204.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/204.1 MB 10.8 MB/s eta 0:00:19
   - -------------------------------------- 5.2/204.1 MB 13.3 MB/s eta 0:00:15
   - -------------------------------------- 8.9/204.1 MB 15.1 MB/s eta 0:00:13
   -- ------------------------------------- 12.3/204.1 MB 15.7 MB/s eta 0:00:13
   --- ------------------------------------ 16.5/204.1 MB 16.6 MB/s

In [2]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

ModuleNotFoundError: No module named 'pandas'

In [3]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [49]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [5]:
from sentence_transformers import SentenceTransformer

C:\Users\dhava\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

In [7]:
vectors.shape

(8, 768)

In [8]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [9]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [10]:
index.add(vectors)

In [11]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000252CF6123D0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [64]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [66]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [67]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [68]:
distances, I = index.search(new_vec, k=2)
distances

array([[1.3844836, 1.4039096]], dtype=float32)

In [69]:
I

array([[3, 2]], dtype=int64)

In [70]:
I.tolist()

[[3, 2]]

In [71]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [72]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [73]:
search_query

'I want to buy a polo t-shirt'

You can see that the two results from the dataframe are similar to a search_query